In [3]:
#Set up DrWatson
cd("/home/jm2386/Active_Lattice/")
using DrWatson
@quickactivate "Active_Lattice"
# Load relevant fuctions
include("/home/jm2386/Active_Lattice/src/pm_pde_functions.jl");
include("/home/jm2386/Active_Lattice/src/Hetrocline.jl");
include("/home/jm2386/Active_Lattice/src/plot_functions.jl");

function animate_phase_pdes_pm(param,t_saves,fa_saves,fp_saves; frames = 99, ϕg = 0.0, ϕl = 1.0)
    @unpack name, λ, ρa, ρp, Nx, Nθ, δt, Dθ, χ, γ = param
    fig, axs = plt.subplots(2, 1, figsize=(10,10))
    N = length(t_saves)
    function makeframe(i)
        clf()
        ax1 = fig.add_subplot(211)
        ax2 = fig.add_subplot(212)
        axs = ax1, ax2
        vid_pde_plot_pm(fig, axs, param, t_saves, fa_saves, fp_saves, i+1)
        binod_liquid = ax1.plot(t_saves, ϕl*ones(N), color = "black", linestyle = "--", label = "_liquid phase")
        binod_gas = ax1.plot(t_saves, ϕg*ones(N), color = "black",linestyle = "--", label = "_gas phase")
        return fig
    end
    interval = 5*Int64(round(20000/frames))
    myanim = anim.FuncAnimation(fig, makeframe, frames=frames, interval=interval)
    # Convert it to an MP4 movie file and saved on disk in this format.
    T = t_saves[Int64(round((frames+1)))]
    pathname = "/store/DAMTP/jm2386/Active_Lattice/plots/vids/pde_phase_vids/$(name)/active=$(ρa)_passive=$(ρp)_lamb=$(λ)";
    mkpath(pathname)
    filename = "/store/DAMTP/jm2386/Active_Lattice/plots/vids/pde_phase_vids/$(name)/active=$(ρa)_passive=$(ρp)_lamb=$(λ)/time=$(round(T; digits = 5))_Nx=$(Nx)_Nθ=$(Nθ)_active=$(ρa)_passive=$(ρp)_lamb=$(λ).mp4";
    myanim[:save](filename, bitrate=-1, dpi= 100, extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
end

function make_phase_video_pm(param; skip_factor = 10, ϕg = 0.0, ϕl = 1.0)
    @unpack T, save_interval = param
    save_interval = save_interval*skip_factor
    t_saves, fa_saves, fp_saves = load_pdes_pm(param,T; save_interval = save_interval)
    frames = Int64(length(t_saves))
    animate_phase_pdes_pm(param,t_saves,fa_saves,fp_saves; frames = frames-1,ϕg =ϕg, ϕl = ϕl)
end

function vid_pde_plot_pm(fig::Figure, axs, param::Dict{String,Any}, t_saves, fa_saves, fp_saves, i)
    @unpack Nx, Nθ, ρa, ρp, χ, Dθ, Dx, k, γ,Pe = param
    ρa_saves, ρp_saves = deepcopy(spatial_density_pm.(fa_saves)), deepcopy(fp_saves)

    push!(ρa_saves[i], ρa_saves[i][1])
    push!(ρp_saves[i], ρp_saves[i][1])

    ρsum = ρp_saves[i]+ρa_saves[i]

    axs[1].plot((0:1:Nx)/Nx,ρa_saves[i], color = "red", label = L"\rho^a")
    axs[1].plot((0:1:Nx)/Nx,ρsum, color = "black", label = L"\rho")
    axs[1].plot((0:1:Nx)/Nx,ρp_saves[i], color = "blue", label = L"\rho^p")

    axs[1].xaxis.set_ticks(0.:0.2:1.0)
    axs[1].xaxis.set_tick_params(labelsize=15)
    axs[1].yaxis.set_tick_params(labelsize=15)
    rhomax = maximum(maximum(ρa_saves))+maximum(maximum(ρp_saves))
    axs[1].axis([0., 1., 0. , rhomax])
    #axs[1].axis([0., 1., min(minimum(minimum(ρa_saves)),minimum(minimum(ρp_saves))),maximum(maximum( ρa_saves+ρp_saves ))])
    axs[1].set_xlabel(L"x",fontsize=20)
    #axs[1].set_ylabel(L"\rho,",fontsize=20)
    title = latexstring("\$ \\ell = $(round(1/sqrt(Dθ); digits = 2)), \\chi = $(χ), \\phi = $(ρa+ρp), \\mathrm{Pe} = $(round(Pe; digits = 3)), t = $(round(t_saves[i]; digits = 3))\$")
    axs[1].set_title(title,fontsize=20)

    mat1 = zeros(1, Nx+1)
    mat2= zeros(1, Nx+1)
    mags = mag_pm(fa_saves[i]; Nθ = Nθ)
    push!(mags,mags[1])
    mat1[1, :] = mags
    mat2[1, :] = mags.*(-ρsum.+1)

    #colmap = PyPlot.plt.cm.seismic
    colmap = PyPlot.plt.cm.PRGn
    norm1 = matplotlib.colors.Normalize(vmin= -rhomax*0.5 , vmax= rhomax*0.5) 
    #norm1 = matplotlib.colors.Normalize(vmin= -maximum(abs.(mags)) , vmax= maximum(abs.(mags)) )
    #norm2 = matplotlib.colors.Normalize(vmin= minimum(mags/10) , vmax= maximum(mags)/10 )

    axs[2].matshow(mat1; norm = norm1,  cmap = colmap, extent = [0., 1., 0., 0.1])
    #axs[3].matshow(mat2; norm = norm2,  cmap = colmap, extent = [0., 1., 0., 0.1])

    axs[2].set_aspect(1.)
    #axs[3].set_aspect(1.)

    axs[2].xaxis.set_ticks(0.:0.2:1.0)
    axs[2].yaxis.set_ticks([])
    axs[2].xaxis.set_tick_params(labelsize=15)
    axs[2].xaxis.tick_bottom()
    #ax.set_title(L"\Re{ \lambda_n^\mathrm{max}} = 0",fontsize=20)
    #ax.set_xlabel(L"x",fontsize=20)

    axs[2].set_ylabel(L"\mathbf{p}", fontsize=20, rotation=0)
    axs[2].yaxis.set_label_coords(-.05, .5)

    lines, labels = axs[1].get_legend_handles_labels()
    fig.tight_layout()
    ldg = fig.legend(lines, labels, loc = "center right", fontsize=20, bbox_to_anchor = (0.25, 0.25, 1, 1),
    bbox_transform = plt.gcf().transFigure)

    return fig
end

   Resolving package versions...


  No Changes to `/mhome/damtp/s/jm2386/Active_Lattice/Project.toml`
  No Changes to `/mhome/damtp/s/jm2386/Active_Lattice/Manifest.toml`


vid_pde_plot_pm (generic function with 1 method)

In [6]:
# parameters
Pe = 7.5;
initial_Δ = 1e-4;
max_iter = 40;
tol = 1e-3;
atol = 1e-12;
rho_max = (1-10e-20);

In [7]:
#quick solve
Pe = 7.5;
γ_limit = find_gamma_limit(;Pe = Pe, initial_Δ = initial_Δ, γ_max = 100.)
γ = 1.9 # γ_limit-0.00001
find_sol, lower_limits, upper_limits = colapse_sol_interval(;Pe = Pe, γ = γ, rho_max = rho_max, initial_Δ = initial_Δ, max_iter = max_iter, tol = tol, atol = atol)
gamma_converter(γ, lower_limits[1])

(0.4561127797358473, 0.2576307885461776)

In [13]:
# set parameters
ρ = 0.3+0.5
χ = 0.5/ρ
Pe = 7.5
pert = "rand"
    T  = 4.0
    δ  = 1e-2
    save_interval = 0.01
    Dx = 1. 
    Dθ = 10000.0
    Nx = 512
    Nθ = 2
    max_steps = 1e7
name = "pm_pde_run_2_δ=$(δ)_l=$(1/sqrt(Dθ))"
#
param = pde_param_pm(; name = name, 
                                ρ = ρ, Pe = Pe, χ = χ, T = T, 
                                Dθ = Dθ, δt = 1e-5, Nx = Nx, Nθ = Nθ, 
                                save_interval = save_interval, max_steps =  max_steps,
                                pert = pert, δ = δ,
                        )






Dict{String, Any} with 31 entries:
  "λs"            => 20.0:20.0:100.0
  "T"             => 4.0
  "video_length"  => 10000.0
  "save_interval" => 0.01
  "max_runs"      => 6
  "Pe"            => 7.5
  "λ_step"        => 10.0
  "δ"             => 0.01
  "Nθ"            => 2
  "name"          => "pm_pde_run_2_δ=0.01_l=0.01"
  "δt"            => 1.0e-5
  "λ"             => 750.0
  "Dθ"            => 10000.0
  "cbar_max"      => 1.0
  "Dx"            => 1.0
  "cbar_min"      => 0.0
  "Nx"            => 512
  "k"             => 20
  "pert"          => "rand"
  ⋮               => ⋮

In [14]:
@unpack T, save_interval, max_steps, pert, δ = param
density = initialize_density_pm(param)
perturb_pde_pm!(param, density; δ = 1e-4, pert = "pm_lin")
run_pde_until_pm!(param,density,T; save_on = true, max_steps = max_steps, save_interval = save_interval)

10000000

In [8]:
#make vid
make_phase_video_pm(param; skip_factor = 10)